# 0.0 Imports

In [3]:
import pandas as pd
import inflection
import math
import numpy as np

#### 0.1 Helper Fuctions

#### Loading Data

In [4]:
df_sales_raw = pd.read_csv('/home/IvanBertaci/Documentos/CIENCIAS_DADOS/COMUNIDADEDS/DS_PRODUCAO/ds_em_producao-main/REPOS/train.csv', low_memory=False)
df_store_raw = pd.read_csv('/home/IvanBertaci/Documentos/CIENCIAS_DADOS/COMUNIDADEDS/DS_PRODUCAO/ds_em_producao-main/REPOS/store.csv', low_memory=False)

# merge

df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

# 1.0 Descrição dos dados

In [5]:
df1 = df_raw.copy()

#### 1.1 Rename Columns

In [6]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

# rename columns

df1.columns = cols_new

#### 1.2 Data Dimensions

In [7]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

Number of Rows: 1017209
Number of Cols: 18


#### 1.3 Data Types

In [8]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

#### 1.4 Check NA

In [9]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

#### 1.5 Fillout NA

In [14]:
#competition_distance 
# Este código converte NA em uma distância de 20000 metros. As NA aconteceram, pois não havia empresas concorrente próximas, desta forma, após verificar qual era a distancia maxima - df1[competition_distance].max(), foi inserido um número maior - 200000, para não ser reconhecido como NA, assim a coluna agora está zerada

df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

#competition_open_since_month
# Este código converte NA para a coluna 'date'. A coluna 'date' é a data de fundação da loja, assim é possível partir do pressuposto que a loja já foi criada com algum concorrente por perto.

df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#competition_open_since_year
# Este código converte NA para a coluna 'date'. A coluna 'date' é a data de fundação da loja, assim é possível partir do pressuposto que a loja já foi criada com algum concorrente por perto.

df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

#promo2
#promo2_since_week
# Este código converte NA para a coluna 'date'.

df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

#promo2_since_year
# Este código converte NA para a coluna 'date'.

df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

#promo_interval
# Este código converte NA em meses

month_map = {1: 'Jan',  2: 'Fev',  3: 'Mar',  4: 'Apr',  5: 'May',  6: 'Jun',  7: 'Jul',  8: 'Aug',  9: 'Sep',  10: 'Oct', 11: 'Nov', 12: 'Dec'}

df1['promo_interval'].fillna(0, inplace=True )

df1['month_map'] = df1['date'].dt.month.map( month_map )

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split( ',' ) else 0, axis=1 )

In [15]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64

In [17]:
df1.sample(20).T

,316126,532391,863924,517140,855863,294847,530914,354060,248713,931557,527322,485792,476713,74261,328243,1005891,713699,655135,240863,832940
store,329,207,585,566,329,593,960,964,222,203,713,438,279,672,286,947,885,301,889,821
day_of_week,2,2,6,2,6,4,4,4,4,1,7,2,3,2,3,5,1,4,6,6
date,2014-10-07 00:00:00,2014-03-11 00:00:00,2013-05-18 00:00:00,2014-03-25 00:00:00,2013-05-25 00:00:00,2014-10-30 00:00:00,2014-03-13 00:00:00,2014-08-28 00:00:00,2014-12-18 00:00:00,2013-03-18 00:00:00,2014-03-16 00:00:00,2014-04-22 00:00:00,2014-04-30 00:00:00,2015-05-26 00:00:00,2014-09-24 00:00:00,2013-01-11 00:00:00,2013-09-30 00:00:00,2013-11-21 00:00:00,2014-12-27 00:00:00,2013-06-15 00:00:00
sales,7093,9272,4684,4247,4391,10471,4594,7704,6831,7477,0,4476,12508,8727,3158,8372,4236,5803,3791,4813
customers,629,1214,802,517,410,1427,436,1287,690,839,0,515,922,1290,324,1103,613,614,373,641
open,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
promo,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0
state_holiday,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
school_holiday,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
store_type,a,a,d,a,a,a,d,a,a,c,a,d,d,c,a,a,a,a,d,a
